In [1]:
%matplotlib inline
import pandas as pd
import sklearn
import numpy as np
from IPython.display import display
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
from matplotlib import rcParams

## Change The Parameters HERE!!! 

In [2]:
input_csv_path = '../../data/the_standard_data.csv'
output_csv_path = '../../data/the_standard_data_with_features.csv'

In [3]:
data = pd.read_csv(input_csv_path)

In [11]:
data.head()

,Unnamed: 0,headline,date,time,view,like,share,comment,category,tag,...,sim_worst,tag_top_pop_sum,tag_top_pop_mean,num_tag_pop,tag_top_worst_sum,tag_top_worst_mean,num_tag_worst,numTag,month,year
0,0,สุดารัตน์การันตี มิ่งขวัญไม่เสียสัจจะ เชื่อประ...,2019-04-13,NaN,368,NaN,NaN,NaN,"Thailand,Politics","พรรคเพื่อไทย,สุดารัตน์ เกยุราพันธุ์,มิ่งขวัญ แ...",...,0.722237,10,2,4,9,4,2,4,4,2019
1,1,สโมสรฟุตบอลซาน ลอเรนโซฯ ในอาร์เจนตินา ประกาศเซ...,2019-04-13,NaN,22,NaN,NaN,NaN,Sport,"Argentina,San Lorenzo de Almagro",...,1.000000,0,0,0,4,2,2,2,4,2019
2,2,ภาพข่าวยอดเยี่ยมประจำวันที่ 7-13 เมษายน 2562,2019-04-13,NaN,31,NaN,NaN,NaN,"World,Thailand",ภาพข่าวยอดเยี่ยม,...,1.000000,0,0,0,16,16,1,1,4,2019
3,3,สงกรานต์สีลม สีสันความสนุก กิจกรรมดับร้อนของคน...,2019-04-13,NaN,280,NaN,NaN,NaN,Thailand,เทศกาลสงกรานต์,...,0.640314,1,1,1,25,25,1,1,4,2019
4,4,14 April 2003 – โครงการศึกษาจีโนมของมนุษย์เสร็...,2019-04-14,NaN,56,NaN,NaN,NaN,On this day,"พันธุกรรม,วิทยาศาสตร์",...,1.000000,0,0,0,2,1,2,2,4,2019


## Tokenize

In [92]:
from pythainlp.tokenize import word_tokenize
from pythainlp.corpus import thai_stopwords
from stop_words import get_stop_words
import string

In [93]:
en_stop = get_stop_words('en')
num = [str(i) for i in range(1000)]
stop_words = list(thai_stopwords())
stop_words+=[' ','']
stop_words+=string.punctuation

In [94]:
def getToken(text):
    res = word_tokenize(text,engine='deepcut')
    res = [str(i).encode('utf-8') for i in res if i not in en_stop and i not in num and i.strip() not in stop_words]
    return res

In [95]:
data['token'] = data['headline'].apply(getToken)

In [96]:
data['numToken'] = data['token'].apply(lambda x : len(x))

In [97]:
data['numChar'] = data['token'].apply(lambda x: len(''.join([i.decode('utf-8') for i in x])))

In [98]:
data['token'] = data['token'].apply(lambda x: [i.decode('utf-8') for i in x])

In [99]:
data['token'] = data['token'].apply(lambda x : [i.replace('“,', '') for i in x])
data['token'] = data['token'].apply(lambda x : [i.replace('”', ',') for i in x])
data['token'] = data['token'].apply(lambda x : [i.replace(',“', ',') for i in x])
data['token'] = data['token'].apply(lambda x : [i.replace(',,', ',') for i in x])
data['token'] = data['token'].apply(lambda x : [i.replace('“', ',') for i in x])

### Clean Junk

## Date & Timedelta

In [100]:
data['date'] = data['date'].apply(lambda x : x.split('/')[0]
                                            +'/'+x.split('/')[1]+'/'+str(int(x.split('/')[2])-543))
data['date'] = pd.to_datetime(data['date'],format="%d/%m/%Y")
data['timeDelta'] = pd.to_datetime('20/04/2019',format="%d/%m/%Y")- data['date']
data['timeDelta'] = data['timeDelta'].apply(lambda x: int(str(x).split(' ')[0]))

In [ ]:
data['month'] = data['date'].apply(lambda x: x.month)
data['year'] = data['date'].apply(lambda x: x.year)

In [10]:
# data['month'] = data['date'].apply(lambda x: int(x.split('-')[1]))
# data['year'] = data['date'].apply(lambda x: int(x.split('-')[0]))

## POS Count

In [101]:
from pythainlp.tag import pos_tag

In [102]:
data['num_noun'] = 0
data['num_pronoun'] = 0
data['num_verb'] = 0
data['num_preverb'] = 0
data['num_determiner'] = 0
data['num_adverb'] = 0
data['num_classifier'] = 0
data['num_conjunction'] = 0
data['num_preposition'] = 0
data['num_interjunction'] = 0
data['num_prefix'] = 0
data['num_ending'] = 0
data['num_negator'] = 0
data['num_punctuation'] = 0

In [108]:
for index, row in data.iterrows():
    pos = pos_tag([i.replace("\''",'') for i in row['token']],engine='artagger')
    for token,p in pos:
        if p[0] =='NEG':
            data.at[index,'NEG'] += 1
        elif p[0] =='N':
            data.at[index,'num_noun'] += 1
        elif p[0] =='PUNC':
            data.at[index,'num_punctuation'] += 1
        elif p[0] =='P':
            data.at[index,'num_pronoun'] += 1
        elif p[0] =='V':
            data.at[index,'num_verb'] += 1
        elif p[0] =='X':
            data.at[index,'num_preverb'] += 1
        elif p[0] =='D':
            data.at[index,'num_determiner'] += 1
        elif p[0] =='A':
            data.at[index,'num_adverb'] += 1
        elif p[0] =='C':
            data.at[index,'num_classifier'] += 1
        elif p[0] =='J':
            data.at[index,'num_conjunction'] += 1
        elif p[0] =='R':
            data.at[index,'num_preposition'] += 1
        elif p[0] =='I':
            data.at[index,'num_interjunction'] += 1
        elif p[0] =='F':
            data.at[index,'num_prefix'] += 1
        elif p[0] =='E':
            data.at[index,'num_ending'] += 1

KeyError: "''"

In [ ]:
data['nnoun_per_ntoken'] = data['num_noun']/data['numToken']
data['nverb_per_ntoken'] = data['num_verb']/data['numToken']
data['nadverb_per_ntoken'] = data['num_adverb']/data['numToken']

## Tag / Category

In [ ]:
unique_category = set()
for i in data['category']:
    for j in str(i).split(','):
        unique_category.add(j.strip())

In [ ]:
for i in list(unique_category):
    data['is_'+i] = data.apply(lambda row : int(i in str(row['category'])),axis=1)

In [ ]:
data['tag_pop_sum'] = 0
data['tag_pop_mean'] = 0

In [ ]:
tag_dict = dict()
for index, row in data.iterrows():
    tag = str(row['tag']).split(',')
    for t in tag:
        if t not in tag_dict.keys():
            tag_dict[t] = 1
        else:
            tag_dict[t] += 1

In [ ]:
for index, row in data.iterrows():
    tag = str(row['tag']).split(',')
    pop_list = [tag_dict[t] for t in tag if t in tag_dict.keys()]
    data.at[index,'tag_pop_sum'] = sum(pop_list)
    if len(pop_list) > 0:
        data.at[index,'tag_pop_mean'] = sum(pop_list)/len(pop_list)

## NER Count

In [39]:
from pythainlp.tag.named_entity import ThaiNameTagger
ner = ThaiNameTagger()

In [40]:
thai_abb = pd.read_csv('../Eye/thai_abb.csv')

In [41]:
abb_set = set(thai_abb['อักษรย่อ'])

In [42]:
data['num_date'] = 0
data['num_email'] = 0
data['num_law'] = 0
data['num_len'] = 0
data['num_location'] = 0
data['num_money'] = 0
data['num_organization'] = 0
data['num_percent'] = 0
data['num_person'] = 0
data['num_phone'] = 0
data['num_time'] = 0
data['num_url'] = 0

In [43]:
for index, row in data.iterrows():
    token = row['token']
    for t in token:
        if t in abb_set:
            t = list(thai_abb[thai_abb['อักษรย่อ']==t]['ชื่อเต็ม'])[0]
        elif t.split('.')[0] in abb_set:
            t = list(thai_abb[thai_abb['อักษรย่อ']==t.split('.')[0]]['ชื่อเต็ม'])[0]
        NER = ner.get_ner(t)
        if len(NER) > 1:
            word,pos,n = NER[1]
        else:
            word,pos,n = NER[0]  
        if n == 'I-DATE':
            data.at[index,'num_date'] += 1
        elif n == 'I-EMAIL':
            data.at[index,'num_email'] += 1
        elif n == 'I-LAW':
            data.at[index,'num_law'] += 1
        elif n == 'I-LEN':
            data.at[index,'num_len'] += 1
        elif n == 'I-LOCATION':
            data.at[index,'num_location'] += 1
        elif n == 'I-MONEY':
            data.at[index,'num_money'] += 1
        elif n == 'I-ORGANIZATION':
            data.at[index,'num_organization'] += 1
        elif n == 'I-PERCENT':
            data.at[index,'num_percent'] += 1
        elif n == 'I-PERSON':
            data.at[index,'num_person'] += 1
        elif n == 'I-PHONE':
            data.at[index,'num_phone'] += 1
        elif n == 'I-TIME':
            data.at[index,'num_time'] += 1
        elif n == 'I-URL':
            data.at[index,'num_url'] += 1         

## Thai Names

In [44]:
thai_name = pd.read_csv('../Eye/ThaiName.txt',sep=" ",header=None)
thai_name.columns = ["firstName", "lastName"]
thai_name.dropna(inplace=True)
thai_name_f = pd.read_csv('../Eye/ThaiName-female.txt',sep=" ",header=None)
thai_name_f.columns = ["firstName"]
thai_name_f.dropna(inplace=True)
thai_name_m = pd.read_csv('../Eye/ThaiName-man.txt',sep=" ",header=None)
thai_name_m.columns = ["firstName"]
thai_name_m.dropna(inplace=True)

In [45]:
thaiNameSet = set(thai_name_f['firstName']).union(set(thai_name_m['firstName']))

In [46]:
data['num_person_2'] = 0

In [47]:
reserve_name = ['สมัคร','ชวน','พายุ','เลื่อน','อำนาจ','สัญญา','นิยม']

In [48]:
for index, row in data.iterrows():
    token = row['token']
    for t in token : 
        if t in thaiNameSet and t not in reserve_name:
            data.at[index,'num_person_2'] += 1   

## Sentiment

In [49]:
pos_df1 = pd.read_csv('../Eye/sentiment_data/positive_new.txt', header=None)
pos_df1.columns = ["pos_word"]
pos_df1.dropna(inplace=True)
pos_df2 = pd.read_csv('../Eye/sentiment_data/positive__verbs.txt', header=None)
pos_df2.columns = ["pos_word"]
pos_df2.dropna(inplace=True)
pos_df3 = pd.read_csv('../Eye/sentiment_data/positive__adjectives.txt', header=None)
pos_df3.columns = ["pos_word"]
pos_df3.dropna(inplace=True)

In [50]:
neg_df1 = pd.read_csv('../Eye/sentiment_data/negative_new.txt', header=None)
neg_df1.columns = ["neg_word"]
neg_df1.dropna(inplace=True)
neg_df2 = pd.read_csv('../Eye/sentiment_data/negative_verbs.txt', header=None)
neg_df2.columns = ["neg_word"]
neg_df2.dropna(inplace=True)
neg_df3 = pd.read_csv('../Eye/sentiment_data/negative_adjectives.txt', header=None)
neg_df3.columns = ["neg_word"]
neg_df3.dropna(inplace=True)

In [51]:
pos_word_set = set(pos_df1['pos_word']).union(set(pos_df2['pos_word'])).union(set(pos_df3['pos_word']))
neg_word_set = set(neg_df1['neg_word']).union(set(neg_df2['neg_word'])).union(set(neg_df3['neg_word']))

In [52]:
data['num_pos'] = 0
data['num_neg'] = 0

In [53]:
for index, row in data.iterrows():
    token = row['token']
    for t in token : 
        if t in pos_word_set:
            data.at[index,'num_pos'] += 1  
        elif t in neg_word_set:
            data.at[index,'num_neg'] += 1  

## Tag

In [64]:
data['tag_pop_sum'] = 0
data['tag_pop_mean'] = 0
data['tag_top_pop_sum'] = 0
data['tag_top_pop_mean'] = 0
data['num_tag_pop'] = 0
data['tag_top_worst_sum'] = 0
data['tag_top_worst_mean'] = 0
data['num_tag_worst'] = 0

In [80]:
data['numTag'] = data.apply(lambda x: len(str(x['tag']).split(',')),axis = 1)

In [66]:
for index, row in data.iterrows():
    tag = str(row['tag']).split(',')
    pop_list = [tag_dict[t] for t in tag if t in tag_dict.keys()]
    data.at[index,'tag_pop_sum'] = sum(pop_list)
    if len(pop_list) > 0:
        data.at[index,'tag_pop_mean'] = sum(pop_list)/len(pop_list)

In [67]:
top_news = pd.DataFrame(data.sort_values('view',ascending=False)[:1000])
worst_news = pd.DataFrame(data.sort_values('view',ascending=True)[:1000])

In [81]:
tag_dict_top = dict()
for index, row in top_news.iterrows():
    tag = str(row['tag']).split(',')
    for t in tag:
        if t not in tag_dict_top.keys():
            tag_dict_top[t] = 1
        else:
            tag_dict_top[t] += 1

In [70]:
tag_dict_worst = dict()
for index, row in worst_news.iterrows():
    tag = str(row['tag']).split(',')
    for t in tag:
        if t not in tag_dict_worst.keys():
            tag_dict_worst[t] = 1
        else:
            tag_dict_worst[t] += 1

In [71]:
for index, row in data.iterrows():
    tag = str(row['tag']).split(',')
    pop_list = [tag_dict_top[t] for t in tag if t in tag_dict_top.keys()]
    data.at[index,'num_tag_pop'] = len(pop_list)
    data.at[index,'tag_top_pop_sum'] = sum(pop_list)
    if len(pop_list) > 0:
        data.at[index,'tag_top_pop_mean'] = sum(pop_list)/len(pop_list)

In [73]:
for index, row in data.iterrows():
    tag = str(row['tag']).split(',')
    worst_list = [tag_dict_worst[t] for t in tag if t in tag_dict_worst.keys()]
    data.at[index,'num_tag_worst'] = len(worst_list)
    data.at[index,'tag_top_worst_sum'] = sum(worst_list)
    if len(worst_list) > 0:
        data.at[index,'tag_top_worst_mean'] = sum(worst_list)/len(worst_list)

## Similarity

In [55]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
from pythainlp.word_vector import sentence_vectorizer

/home/arintrongs2/.pyenv/versions/3.6.0/lib/python3.6/site-packages/smart_open/ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')


In [56]:
def findMaxSim(embed1, headline_embed):
    a = np.asarray([cosine_similarity(embed1,x) for x in headline_embed])
    return a.max()

In [57]:
top_headline = np.asarray(pd.DataFrame(data.sort_values('view',ascending=False)[:1000]['headline'])['headline'])
top_headline_embed = np.asarray([sentence_vectorizer(x) for x in top_headline])
worst_headline = np.asarray(pd.DataFrame(data.sort_values('view',ascending=True)[:1000]['headline'])['headline'])
worst_headline_embed = np.asarray([sentence_vectorizer(x) for x in worst_headline])

### ** Becareful this cell runs fucking long**

In [58]:
for index, row in data.iterrows():
    sent = row['headline']
    sent_embed = sentence_vectorizer(sent)
    data.at[index,'sim_top'] = findMaxSim(sent_embed,top_headline_embed)
    data.at[index,'sim_worst'] = findMaxSim(sent_embed,worst_headline_embed)

## Save to .CSV

In [12]:
data.to_csv(output_csv_path)